In [1]:
import numpy as np
from torchvision.transforms import Compose, ToTensor
import torch
import torch.nn as nn
import torch.nn.functional as F
#import torch.optim as optim
#import torchvision
from torch.utils.data import DataLoader
#import torch
import os 
#import librosa
from torch.optim import Adadelta
#from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import gc
import pytorch_lightning as pl
import time
import pickle
import optuna

from utils_mgr import readheavy, get_stft, clip_audio, clip_stft, DataAudio


print("let's start")

##tensorboard --logdir=lightning_logs/ 
# to visualize logs




/home/alberto/anaconda3/envs/mgr2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-03 18:57:04.864439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


let's start


In [2]:

n_train = 1


print("reading train")
#Convert Audio into stft data
#train = readheavy("training",1,f"Audio/")

train =  np.load(f"Data/Audio/training_{n_train}.npy", allow_pickle = True)


# take each song and splits it into clips of n_samples 
# creates 
print("making clips")
train_clip = clip_audio(train, 65_536) 

reading train
making clips


In [3]:
train_clip[0][0].shape

(65536,)

In [4]:
torch.Tensor(train_clip[0][0])

tensor([ 1.4901e-08,  2.6077e-08,  3.7253e-09,  ..., -5.7922e-03,
         4.2363e-04,  1.3559e-02])

In [5]:

n_train = 1


print("reading train")
#Convert Audio into stft data
#train = readheavy("training",1,f"Audio/")

train =  np.load(f"Data/Audio/training_{n_train}.npy", allow_pickle = True)

valid = readheavy("validation",2,"Data/Audio/")

# take each song and splits it into clips of n_samples 
# creates 
print("making clips")
train_clip = clip_audio(train, 65_536)   #65536 is the closest power of 2 to reproduce clips of 3s
print("making clips")
valid_clip = clip_audio(valid, 65_536)    

transforms = Compose([ torch.Tensor, ]) # Normalize(0,1) is not necessary for stft data

print('Dimensione dataset: ', train_clip.shape)

train_dataset = DataAudio(data=train_clip,transform=transforms)
valid_dataset = DataAudio(data=valid_clip,transform=transforms)

del train_clip
del valid_clip

gc.collect()

#Creation of dataloader classes
print("train dataloader")
train_dataloader = DataLoader(train_dataset, 64, shuffle=True, num_workers=os.cpu_count())
print("valid dataloader")
valid_dataloader = DataLoader(valid_dataset, 64, shuffle=False, num_workers=os.cpu_count())
print("data loaded")

del train_dataset
del valid_dataset
gc.collect()



reading train
making clips
making clips
Dimensione dataset:  (7581, 2)
train dataloader
valid dataloader
data loaded


0

In [6]:
train_dataloader.dataset.x[0]


array([ 1.4901161e-08,  2.6077032e-08,  3.7252903e-09, ...,
       -5.7921670e-03,  4.2362884e-04,  1.3559222e-02], dtype=float32)

In [7]:
for x,y in train_dataloader:
    print(x.shape)
    break

torch.Size([64, 65536])


: 